#### Relevant imports

In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time

#### Initialize key variables

In [2]:
max_threads_open = 3
items_dict = {}
shared_results = {
    'expected_items': 0,
    'should_be_added': 0
}
categories_to_scrape = [
    ("https://www.terminalx.com/men/shirts/sleevless-tops", "Topwear", "Casual"),
    ("https://www.terminalx.com/men/shirts/tshirts", "Topwear", "Party"),
    ("https://www.terminalx.com/men/shirts/dress-shirts", "Topwear", "Party"),
    ("https://www.terminalx.com/men/shirts/polo", "Topwear", "Party"),
    ("https://www.terminalx.com/men/shirts/sports-tops", "Topwear", "Casual"),
    ("https://www.terminalx.com/men/pants/jeans", "Bottomwear", "Casual"),
    ("https://www.terminalx.com/men/pants/short-pants", "Bottomwear", "Casual"),
    ("https://www.terminalx.com/men/pants/long-pants", "Bottomwear", "Party"),
    ("https://www.terminalx.com/men/pants/sport-pants", "Bottomwear", "Party"),
    ("https://www.terminalx.com/men/pants/joggers", "Bottomwear", "Casual"),
    ("https://www.terminalx.com/men/shoes/sneakers-shoes", "Footwear", "Casual"),
    ("https://www.terminalx.com/men/shoes/sports-shoes", "Footwear", "Casual"),
    ("https://www.terminalx.com/men/shoes/flip-flops", "Footwear", "Casual"),
    ("https://www.terminalx.com/men/shoes/elegant", "Footwear", "Formal"),
    ("https://www.terminalx.com/men/shoes/boots", "Footwear", "Casual"),
    ("https://www.terminalx.com/men/shoes/slippers", "Footwear", "Casual"),
    ("https://www.terminalx.com/men/jackets-coats/coats", "Topwear", "Party"),
    ("https://www.terminalx.com/men/jackets-coats/jackets", "Topwear", "Party"),
    ("https://www.terminalx.com/men/knitwear-sweatshirts/sweatshirts", "Topwear", "Casual"),
    ("https://www.terminalx.com/men/knitwear-sweatshirts/knitwear", "Topwear", "Casual"),
    ("https://www.terminalx.com/men/knitwear-sweatshirts/cardigans", "Topwear", "Party"),
    ("https://www.terminalx.com/women/tops/shirts", "Topwear", "Party"),
    ("https://www.terminalx.com/women/tops/tshirts", "Topwear", "Casual"),
    ("https://www.terminalx.com/women/tops/dress-shirts", "Topwear", "Party"),
    ("https://www.terminalx.com/women/tops/tank-tops", "Topwear", "Casual"),
    ("https://www.terminalx.com/women/tops/sport-tops", "Topwear", "Casual"),
    ("https://www.terminalx.com/women/tops/leotard", "Topwear", "Casual"),
    ("https://www.terminalx.com/women/pants-skirts/jeans", "Bottomwear", "Casual"),
    ("https://www.terminalx.com/women/pants-skirts/long-pants", "Bottomwear", "Party"),
    ("https://www.terminalx.com/women/pants-skirts/leggings", "Bottomwear", "Casual"),
    ("https://www.terminalx.com/women/pants-skirts/shorts", "Bottomwear", "Casual"),
    ("https://www.terminalx.com/women/pants-skirts/joggers", "Bottomwear", "Casual"),
    ("https://www.terminalx.com/women/dresses/maxi", "Bottomwear", "Party"),
    ("https://www.terminalx.com/women/dresses/midi", "Bottomwear", "Party"),
    ("https://www.terminalx.com/women/dresses/short-dresses", "Bottomwear", "Party"),
    ("https://www.terminalx.com/women/dresses/maxiskirts", "Bottomwear", "Party"),
    ("https://www.terminalx.com/women/dresses/midiskirts", "Bottomwear", "Party"),
    ("https://www.terminalx.com/women/dresses/miniskirts", "Bottomwear", "Party"),
    ("https://www.terminalx.com/women/shoes/sandals", "Footwear", "Casual"),
    ("https://www.terminalx.com/women/shoes/sneakers-shoes", "Footwear", "Casual"),
    ("https://www.terminalx.com/women/shoes/flat-shoes", "Footwear", "Party"),
    ("https://www.terminalx.com/women/shoes/sports-shoes", "Footwear", "Casual"),
    ("https://www.terminalx.com/women/shoes/heels", "Footwear", "Party"),
    ("https://www.terminalx.com/women/shoes/slippers", "Footwear", "Casual"),
    ("https://www.terminalx.com/women/shoes/boots", "Footwear", "Party"),
    ("https://www.terminalx.com/women/jackets-coats/jackets", "Topwear", "Formal"),
    ("https://www.terminalx.com/women/jackets-coats/coats", "Topwear", "Formal"),
    ("https://www.terminalx.com/women/knitwear-sweatshirts/sweatshirts", "Topwear", "Casual"),
    ("https://www.terminalx.com/women/knitwear-sweatshirts/knitwear", "Topwear", "Casual"),
    ("https://www.terminalx.com/women/knitwear-sweatshirts/cardigans", "Topwear", "Party")
]

#### Create the scraping functionality for each category

In [3]:
def scrape_category(url, category, occasion):
    options = webdriver.ChromeOptions()
    options.add_argument("--window-position=-3000,0")
    options.add_argument("ignore-certificate-errors")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    driver = webdriver.Chrome(options=options)
    try:
        driver.set_window_size(1920, 1080)
        driver.get(url)
        driver.execute_script("document.body.style.zoom = '90%'")
        num_items_toolbar = driver.find_element(By.CLASS_NAME, "toolbar-number").find_element(By.XPATH, './span[1]')
        num_items = int(num_items_toolbar.text)
        expected = int(num_items_toolbar.text)
        items_in_row = 4
        start = 0
        loaded = 0
        ads = 0
        duplicates = 0
        while start < num_items:
            retry = 0
            max_tries = 4
            end = min(start + items_in_row, num_items)
            if num_items - ads > end:
                WebDriverWait(driver, 20).until(lambda driver: len(driver.find_elements(By.CLASS_NAME, 'listing-product_3mjp')) > end)
                
            while retry < max_tries:
                retry += 1
                try:
                    row_items = driver.find_elements(By.CLASS_NAME, 'listing-product_3mjp')[start:end]
                    if row_items:
                        break  
                except (NoSuchElementException, IndexError):
                    time.sleep(0.1)
                except Exception as e:
                    print(f"An unexpected error occurred: {e}")
                    return
                
            if not row_items:
                print('Unable to retrieve the rest of the elements, moving on to the next category.')
                return
                
            driver.execute_script("arguments[0].scrollIntoView({block: 'start'});", row_items[0])

            for item in row_items:
                start += 1
                loaded += 1
                child_divs = item.find_elements(By.XPATH, './div')
                if not child_divs:
                    # advertisement
                    num_items += 1
                    ads += 1
                    continue
                
                img_wrapper = item.find_element(By.XPATH, './div[2]')
                description = item.find_element(By.XPATH, './div[3]/div')
                img = img_wrapper.find_element(By.TAG_NAME, 'img').get_attribute("src")
                link = img_wrapper.find_element(By.TAG_NAME, 'a').get_attribute("href")
                if not img or not link:
                    # did not load correctly
                    expected -= 1
                    continue

                color = link.split("/")[-1].split("=")[-1]
                code = "TERMINALX-" + link.split("/")[-1].replace("?color=", "")
                if code in items_dict:
                    # duplicate
                    duplicates += 1
                    expected -= 1
                    continue

                c_type = link.split("/")[5]
                brand_wrapper = description.find_element(By.XPATH, './div[2]/span')
                brand = brand_wrapper.text if brand_wrapper else "UNKNOWN"
                price = description.find_element(By.CLASS_NAME, 'final-price_8CiX').text
                title = img_wrapper.get_attribute("title")
                if "women" in url:
                    gender = "women"
                elif "men" in url:
                    gender = "men"

                dict_item = {
                    'code': code,
                    'img': img,
                    'title': title,
                    'price': price,
                    'link': link,
                    'gender': gender,
                    'brand': brand,
                    'category' : category,
                    'occasion': occasion,
                    'type': c_type,
                    'color': color
                }
                items_dict[code] = dict_item

    except Exception as e:
        print(e)
        
    finally:
        print(f"total for url {url}:")
        print(f"total items:  {num_items}")
        print(f"items loaded: {loaded}")
        print(f"ads:          {ads}")
        print(f"duplicates:   {duplicates}")
        print(f"expected:     {expected}")
        print(f"added items:  {loaded - ads - duplicates}\n")
        shared_results['expected_items'] += expected
        shared_results['should_be_added'] += loaded - ads - duplicates
        driver.quit()

#### Run former function for all categories async

In [4]:
with ThreadPoolExecutor(max_workers=max_threads_open) as executor:
    for (url, category, occasion) in categories_to_scrape:
        executor.submit(scrape_category, url, category, occasion)

total for url https://www.terminalx.com/men/shirts/sleevless-tops:
total items:  128
items loaded: 128
ads:          6
duplicates:   0
expected:     122
added items:  122

total for url https://www.terminalx.com/men/shirts/polo:
total items:  264
items loaded: 264
ads:          12
duplicates:   0
expected:     252
added items:  252

total for url https://www.terminalx.com/men/shirts/sports-tops:
total items:  184
items loaded: 184
ads:          8
duplicates:   15
expected:     161
added items:  161

total for url https://www.terminalx.com/men/shirts/dress-shirts:
total items:  694
items loaded: 694
ads:          30
duplicates:   0
expected:     664
added items:  664

Unable to retrieve the rest of the elements, moving on to the next category.
total for url https://www.terminalx.com/men/pants/jeans:
total items:  521
items loaded: 520
ads:          23
duplicates:   1
expected:     497
added items:  496

total for url https://www.terminalx.com/men/pants/short-pants:
total items:  601
ite

#### Add the scraped data to a MongoDB collection

In [5]:
with MongoClient('mongodb://localhost:27017/') as client:
    db = client['Fashion_Stylist']
    collection = db['TERMINALX']
    documents = list(items_dict.values())
    collection.insert_many(documents)
    time.sleep(0.1)
    added = collection.count_documents({})

print(f'initially expected: {shared_results["expected_items"]}')
print(f'should be added:    {shared_results["should_be_added"]}')
print(f'added:              {added}')

initially expected: 26476
should be added:    26466
added:              26466
